# Quantum NN Classification: XOR

In [1]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.3.0


In [2]:
# we will test on the same data we train on
train_x = np.array([[0,0],[0,1],[1,0],[1,1]])
train_labels = np.array([0,1,1,0])

test_x = train_x.copy()
test_labels = train_labels.copy()

In [3]:
train_x.shape

(4, 2)

In [4]:
train_labels

array([0, 1, 1, 0])

# Build Model
We need to ensure that it is non-linear since the xor problem is nonlinear.

We implement a quantum layer from https://towardsdatascience.com/integrating-tensorflow-and-qiskit-for-quantum-machine-learning-7fa6b14d5294

In [5]:
from qiskit.aqua.operators import Z, Y, X
from qiskit.aqua.operators import StateFn
from qiskit.circuit import QuantumRegister, QuantumCircuit

QUBITS = 4
operatorZ = Z ^ Z ^ Z ^ Z
operatorX = X ^ X ^ X ^ X
operatorY = Y ^ Y ^ Y ^ Y

def quantum_layer(initial_parameters):
    # expecting parameters to be a numpy array
    quantumRegister = QuantumRegister(QUBITS)
    quantumCircuit = QuantumCircuit(quantumRegister)
    
    quantumCircuit.h(range(4))

    for i in range(len(initial_parameters)):
        quantumCircuit.ry(initial_parameters[i] * np.pi, i)
   
    psi = StateFn(quantumCircuit)
    
    # two ways of doing the same thing
    expectationX = (~psi @ operatorX @ psi).eval()
    expectationZ = psi.adjoint().compose(operatorZ).compose(psi).eval().real
    expectationY = (~psi @ operatorY @ psi).eval()
    
    expectationZ = np.abs(np.real(expectationZ))
    expectations = [expectationX, expectationY, expectationZ, 
                    expectationX + expectationY + expectationZ] 
    
    # multiply by scalar since can be small values
    expectations = [e * 10 for e in expectations]

    return np.array(expectations)

Integrate quantum layer into keras layer

In [6]:
class Linear(tf.keras.layers.Layer):
    def __init__(self, batch_size=64, units=4,**kwargs):
        super(Linear, self).__init__(**kwargs)  

    def get_config(self):
        config = super(Linear, self).get_config()
        return config

    def call(self, inputs):
        
        if(tf.executing_eagerly()):
            print("Yes!")
            final_output = []
            for i in range(inputs.shape[0]):
                pred = quantum_layer(inputs[i].numpy())
                final_output.append(list(pred))
            return tf.convert_to_tensor(final_output)
        return inputs

Put in our new quantum linear layer

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(4, activation='relu'),
    Linear(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [8]:
# must be true
tf.executing_eagerly()

True

In [9]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss= tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [10]:
model.fit(train_x, train_labels, epochs=1000, verbose=0);

In [11]:
model.predict(test_x)

array([[0.00486276],
       [0.9973995 ],
       [0.99802566],
       [0.00486276]], dtype=float32)

### Evaluate accuracy

In [12]:
test_loss, test_acc = model.evaluate(test_x,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

1/1 - 0s - loss: 0.0036 - accuracy: 1.0000

Test accuracy: 1.0


softmax output of model

In [13]:
model.predict(test_x)

array([[0.00486276],
       [0.9973995 ],
       [0.99802566],
       [0.00486276]], dtype=float32)

label predictions of model

In [14]:
tf.round(model.predict(test_x))

<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
array([[0.],
       [1.],
       [1.],
       [0.]], dtype=float32)>

In [15]:
test_labels

array([0, 1, 1, 0])

Prediction on one input x

In [16]:
x = test_x[0]
print(f"x: {x}")
print(f"shape: {x.shape}")

x: [0 0]
shape: (2,)


In [17]:
# Add the x to a batch where it's the only member.
x = (np.expand_dims(x,0))
print(f"x: {x}")
print(f"shape: {x.shape}")

x: [[0 0]]
shape: (1, 2)


In [18]:
prediction = tf.round(model.predict(x)).numpy()[0][0]
print(f"true: {test_labels[0]}")
print(f"pred: {prediction}")

true: 0
pred: 0.0
